# 前処理大全 Python版

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [93]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from imblearn.under_sampling import RandomUnderSampler

production_tb = pd.read_csv("./data/production.csv")
monthly_index_tb = pd.read_csv("./data/monthly_index.csv")
customer_tb = pd.read_csv("./data/customer.csv")

# 演習問題

In [35]:
production_tb.head(5)

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [34]:
monthly_index_tb.head(5)

,year_month,sales_amount,customer_number
0,2010-01,7191240,6885
1,2010-02,6253663,6824
2,2010-03,6868320,7834
3,2010-04,7147388,8552
4,2010-05,8755929,8171


---
> 006_split/01_under_sampling

- データ不均衡の場合にやるケースがある。
- 機械学習モデルで不均衡を調整する方法もあるが、データを均一にした方が良いケースが多い。
  - モデルによって特性がことなり、使いこなしが難しいためと書いてある。
- またunder samplingも極力やらない。基本は少ないデータをover samplingする。
- ただしover samplingのやりすぎでは劣化するため、over samplingで不均衡を解消できない場合は、多い方をunder samplingすると考える。

---
> 006_split/02_over_sampling

- SMOTEという手法がある。
- SMOTEは似たサンプルを足し合わせて新しいサンプルを生成する。
  - 直線間でのサンプリングとなるため、次元数が大きい場合には、偏りが生じ易くなる。
  - その場合は、under sampleとバギングを組み合わせる方が安定することが多い。
- 書籍の書き方は少し古いため注意が必要。

In [90]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='auto', k_neighbors=5, random_state=777)
blance_data, balance_target = sm.fit_resample(production_tb[['length', 'thickness']], production_tb['fault_flg'])

print(len(balance_target[balance_target]), len(balance_target[balance_target==False]))

948 948
